# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
customers =  orders.groupby('CustomerID').agg(sum_amount_spent = pd.NamedAgg(column = 'amount_spent', aggfunc = sum))
customers.reset_index(level=0, inplace =True)


# Df con la suma del gasto por cliente
customers.head()

,CustomerID,sum_amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [4]:
customers.shape[0]

4339

In [5]:
# obteniendo los percentiles
q_75 = customers['sum_amount_spent'].quantile(0.75)
q_95 = customers['sum_amount_spent'].quantile(0.95)

print('Percentil 75:', q_75)
print('Percentil 95:', q_95)


# Creamos una columna para etiquetar a los clientes, aquellos donde su gasto sea menor al percentil 75, se etiquetará
# como BASIC

customers['group'] = np.where( customers['sum_amount_spent'] < q_75, 'Basic', 
                             np.where(customers['sum_amount_spent'] < q_95, 'Preferred', 'VIP') )

Percentil 75: 1661.6400000000003
Percentil 95: 5840.181999999983


In [6]:
customers.head()

,CustomerID,sum_amount_spent,group
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Basic


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [7]:
# Cruzamos los df de orders 
data_country  = pd.merge(orders[['CustomerID', 'Country']], customers[['CustomerID', 'group']], on='CustomerID', how='inner')

# Quitamos registros totalmente duplicados
data_country.drop_duplicates(inplace=True)

data_country.sort_values(by=['CustomerID'], inplace =True)

# Aplicar value counts sobre country filtrando por clientes VIP
data_country[ data_country.group == 'VIP']['Country'].value_counts()

United Kingdom     177
Germany             10
France               9
Switzerland          3
Spain                2
Portugal             2
Australia            2
EIRE                 2
Japan                2
Sweden               1
Netherlands          1
Cyprus               1
Singapore            1
Channel Islands      1
Denmark              1
Norway               1
Belgium              1
Finland              1
Name: Country, dtype: int64

United Kingdom es el país con más clientes VIP 

In [8]:
data_country[ data_country.group != 'Basic']['Country'].value_counts()

United Kingdom     932
Germany             39
France              29
Belgium             12
Spain                9
Switzerland          9
Norway               7
Portugal             7
Finland              5
Italy                5
Channel Islands      4
Japan                4
Cyprus               4
Australia            4
Austria              3
EIRE                 3
Denmark              3
Sweden               2
Israel               2
Greece               1
Poland               1
Iceland              1
Singapore            1
Malta                1
Netherlands          1
Canada               1
Lebanon              1
Name: Country, dtype: int64

United Kingdom tiene más clientes VIP y Preferentes